## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-03-06-40-16 +0000,bbc,Watch: Australians play in snowy winter wonder...,https://www.bbc.com/news/videos/crm479410ndo?a...
1,2025-08-03-06-23-25 +0000,bbc,Ukraine hits oil depot and Russia strikes home...,https://www.bbc.com/news/articles/ckglyv396ppo...
2,2025-08-03-06-00-57 +0000,wapo,Israel’s support for clans in Gaza puts tribal...,https://www.washingtonpost.com/world/2025/08/0...
3,2025-08-03-05-55-47 +0000,bbc,Pro-Palestine march closes Sydney Harbour Bridge,https://www.bbc.com/news/articles/clydnvw1dg5o...
4,2025-08-03-05-22-51 +0000,nypost,Sydney Sweeney’s voter registration revealed a...,https://nypost.com/2025/08/03/us-news/sydney-s...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2339/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
138,trump,23
54,new,7
259,former,7
514,bar,6
229,city,6


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
44,2025-08-02-22-28-00 +0000,wsj,A watchdog agency opened an investigation into...,https://www.wsj.com/politics/policy/watchdog-a...,72
29,2025-08-03-00-48-00 +0000,wsj,Erika McEntarfer led the Bureau of Labor Stati...,https://www.wsj.com/politics/policy/the-econom...,49
14,2025-08-03-03-00-00 +0000,wsj,Swiss business officials struggle to wrap thei...,https://www.wsj.com/economy/trade/trumps-slap-...,46
33,2025-08-03-00-09-55 +0000,cbc,U.S. watchdog launches investigation into form...,https://www.cbc.ca/news/world/jack-smith-speci...,46
81,2025-08-02-18-38-43 +0000,nyt,Office of Special Counsel Opens Investigation ...,https://www.nytimes.com/2025/08/02/us/politics...,45


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
44,72,2025-08-02-22-28-00 +0000,wsj,A watchdog agency opened an investigation into...,https://www.wsj.com/politics/policy/watchdog-a...
34,35,2025-08-03-00-04-44 +0000,nypost,Cincinnati police chief under intense scrutiny...,https://nypost.com/2025/08/02/us-news/cincinna...
66,32,2025-08-02-20-20-05 +0000,nypost,Bill Maher slams woke mob over Sydney Sweeney ...,https://nypost.com/2025/08/02/us-news/bill-mah...
83,32,2025-08-02-18-35-14 +0000,cbc,"U.S. has 'very, very good plan' to end war in ...",https://www.cbc.ca/news/world/us-plan-ceasefir...
74,31,2025-08-02-19-21-35 +0000,nypost,Three killed in Russia as Ukraine takes aim at...,https://nypost.com/2025/08/02/world-news/three...
107,27,2025-08-02-14-07-39 +0000,wapo,Montana bar shooting leaves 4 dead as authorit...,https://www.washingtonpost.com/nation/2025/08/...
25,25,2025-08-03-01-04-42 +0000,nypost,Virginia city Councilman Lee Vogler faces ‘lon...,https://nypost.com/2025/08/02/us-news/virginia...
127,24,2025-08-02-10-00-00 +0000,latimes,Indian businesses in SoCal brace for 25% tarif...,https://www.latimes.com/business/story/2025-08...
80,23,2025-08-02-18-39-45 +0000,nypost,Islamic Republic appoints ‘Kennedy of Iran’ to...,https://nypost.com/2025/08/02/world-news/islam...
145,22,2025-08-02-09-00-25 +0000,nyt,No Pets Left Behind: How a Dog’s Hurricane Res...,https://www.nytimes.com/2025/08/02/us/trooper-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
